In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [2]:
def limpiar_y_convertir_precio(precio_str):
    tipo_cambio_usd_mxn = 18.66
    # Manejar valores nulos o no válidos
    if pd.isna(precio_str) or not isinstance(precio_str, str):
        return None

    # Detectar el tipo de moneda
    if precio_str.startswith('USD'):
        # Limpiar el string: quitar "USD " y las comas
        valor_numerico_str = precio_str.replace('USD ', '').replace(',', '')
        try:
            valor_numerico = float(valor_numerico_str)
            # Convertir a MXN
            return int(valor_numerico * tipo_cambio_usd_mxn)
        except ValueError:
            return None
    elif precio_str.startswith('MN'):
        # Limpiar el string: quitar "MN " y las comas
        valor_numerico_str = precio_str.replace('MN ', '').replace(',', '')
        try:
            return int(valor_numerico_str)
        except ValueError:
            return None
    else:
        # Si el formato no coincide, devolver un valor nulo
        return None


In [3]:
# Ruta de los directorios que contienen los archivos CSV
directorio_path = ["/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/por_municipio_2025-08-31",
                   "/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/por_municipio_2025-09-14",]


In [4]:

# Lista para almacenar los DataFrames individuales
lista_dfs = []


In [5]:
archivos_csv = []
# Buscar todos los archivos con la extensión .csv en el directorio
for path in directorio_path:
    archivos = glob.glob(os.path.join(path, "*.csv"))
    archivos_csv = archivos_csv + archivos


In [6]:
len(archivos_csv)

30

In [7]:
# Iterar sobre cada archivo encontrado
for archivo in archivos_csv:
    # Leer el archivo CSV en un DataFrame
    df_temp = pd.read_csv(archivo)

    # Añadir el DataFrame temporal a la lista
    lista_dfs.append(df_temp)

In [8]:
df = pd.concat(lista_dfs, ignore_index=True)
print("DataFrame final creado con éxito. Se añadieron los siguientes archivos:")

DataFrame final creado con éxito. Se añadieron los siguientes archivos:


In [9]:
df.columns

Index(['Título', 'Enlace', 'Precio', 'Dirección', 'Características',
       'Municipio de Origen'],
      dtype='object')

In [10]:
df['Municipio de Origen'].value_counts()

Municipio de Origen
Miguel Hidalgo            4679
Cuauhtémoc                3182
Benito Juárez             1812
Cuajimalpa de Morelos     1348
Álvaro Obregón            1122
Coyoacán                   346
Tlalpan                    240
Azcapotzalco               203
Gustavo A. Madero          196
Iztacalco                  107
Venustiano Carranza        105
Iztapalapa                  71
La Magdalena Contreras      69
Tláhuac                     60
Xochimilco                  60
Name: count, dtype: int64

In [11]:
print("\nPrimeras 5 filas del DataFrame final:")
df.head()


Primeras 5 filas del DataFrame final:


,Título,Enlace,Precio,Dirección,Características,Municipio de Origen
0,Departamento en Renta – San Jerónimo (100 m²) ...,https://www.inmuebles24.com/propiedades/clasif...,"MN 24,000","San Jerónimo Lidice\nSan Jerónimo Lídice, La M...","100 m² lote, 2 rec., 2 baños, 1 estac.",La Magdalena Contreras
1,Departamento remodelado. amueblado. frente al ...,https://www.inmuebles24.com/propiedades/clasif...,"MN 12,000","CALLEJON DEL PRADO\nBarrio San Francisco, La M...","50 m² lote, 1 rec., 1 baño, 1 estac.",La Magdalena Contreras
2,Ph en san jeronimo lidice. Departamento con ma...,https://www.inmuebles24.com/propiedades/clasif...,"MN 34,100","BLVD. ADOLFO RUIZ CORTINES, 2775\nSan Jerónimo...","232 m² lote, 3 rec., 4 baños, 2 estac.",La Magdalena Contreras
3,"Se renta casa de 80m2, más 85m2 de jardines. S...",https://www.inmuebles24.com/propiedades/clasif...,"MN 16,000","Magnolia 26\nSan Jerónimo Lídice, La Magdalena...","165 m² lote, 2 rec., 1 baño, 1 estac.",La Magdalena Contreras
4,"Exclusivo conjunto en Pedregal 2, cuenta con c...",https://www.inmuebles24.com/propiedades/clasif...,"MN 26,000","San Marcos 11\nPedregal 2, La Magdalena Contreras","180 m² lote, 3 rec., 2 baños, 2 estac.",La Magdalena Contreras


In [12]:
df.shape

(13600, 6)

In [13]:
# Remove duplicates
df = df.drop_duplicates()

In [14]:
df.shape

(9227, 6)

In [15]:
df.dtypes

Título                 object
Enlace                 object
Precio                 object
Dirección              object
Características        object
Municipio de Origen    object
dtype: object

In [16]:
# Transformar el precio a columna númerica
df['precio_mxn'] = df['Precio'].apply(limpiar_y_convertir_precio)


In [17]:
df[df['Precio'] == 'MN 3,849,092']

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn
693,Con una ubicación privilegiada y excelentes am...,https://www.inmuebles24.com/propiedades/clasif...,"MN 3,849,092","bucareli\nJuárez, Cuauhtémoc","76 m² lote, 2 rec., 2 baños, 1 estac.",Cuauhtémoc,3849092.0


In [18]:
row_to_fix = df['Precio'] == 'MN 3,849,092'

# Re-aplica la función corregida solo a esa fila, dividiendo por 100
df.loc[row_to_fix, 'precio_mxn'] = float(
    df.loc[row_to_fix, 'Precio']
    .iloc[0] # Selecciona el valor de la celda específica
    .replace('MN ', '')
    .replace(',', '')
) / 100


In [19]:
df[df['Precio'] == 'MN 3,849,092']

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn
693,Con una ubicación privilegiada y excelentes am...,https://www.inmuebles24.com/propiedades/clasif...,"MN 3,849,092","bucareli\nJuárez, Cuauhtémoc","76 m² lote, 2 rec., 2 baños, 1 estac.",Cuauhtémoc,38490.92


In [20]:
row_to_fix = df['Precio'] == 'MN 19,250,000'

# Re-aplica la función corregida solo a esa fila, dividiendo por 100
df.loc[row_to_fix, 'precio_mxn'] = float(
    df.loc[row_to_fix, 'Precio']
    .iloc[0] # Selecciona el valor de la celda específica
    .replace('MN ', '')
    .replace(',', '')
) / 1000


In [21]:
df[df['Precio'] == 'MN 19,250,000']

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn
12052,Departamento en Renta con 2 Recámaras con clos...,https://www.inmuebles24.com/propiedades/clasif...,"MN 19,250,000","Vértiz Narvarte\nNarvarte Oriente, Benito Juárez","60 m² lote, 2 rec., 1 baño, 1 estac.",Benito Juárez,19250.0


In [22]:
# Conjunto para almacenar los nombres de características únicos
caracteristicas_unicas = set()

# Expresión regular para encontrar el nombre de la característica
# Busca un número (y opcionalmente m²) seguido del nombre de la característica
patron = re.compile(r'\d+\s*(?:m²)?\s*([a-záéíóúüñ\s\w]+\b)\.?', re.IGNORECASE)

# Iterar sobre cada fila para analizar la columna 'caracteristicas'
for caracteristicas_str in df['Características']:
    if isinstance(caracteristicas_str, str):
        # Encontrar todas las coincidencias en la cadena
        matches = patron.findall(caracteristicas_str)
        
        # Añadir cada característica encontrada al conjunto
        for match in matches:
            # Limpiar el nombre (eliminar espacios y puntos finales)
            nombre_limpio = match.strip().replace('.', '')
            caracteristicas_unicas.add(nombre_limpio)

In [23]:
caracteristicas_unicas

{'a 2 rec',
 'a 80 m² lote',
 'a 95 m² lote',
 'baño',
 'baños',
 'estac',
 'lote',
 'rec',
 'un'}

In [24]:
# Con la revisión de outliers vimos que hay lcoales comerciales con precios altos que no deberíamos estar incluyendo, 
# así que creamos esta flag para identificarlos y excluirlos.
# 1. Usar .str.contains() para buscar las palabras "local comercial" (ignorando mayúsculas y minúsculas)
# 2. Usar .astype(int) para convertir True/False a 1/0
df['es_local_comercial'] = df['Título'].str.contains('local comercial', case=False, na=False).astype(int)

In [25]:
df.es_local_comercial.value_counts()

es_local_comercial
0    9222
1       5
Name: count, dtype: int64

In [26]:
df = df[df.es_local_comercial == 0]

In [27]:
# Borramos la columna porque ya no será necesaria
df = df.drop('es_local_comercial', axis = 1)

In [28]:
# Definimos la leyenda a eliminar
leyenda_a_eliminar = '*Aviso de privacidad sugerencias, quejas, aclaraciones, consultar nuestra página. El precio no incluye el mobiliario, electrodomésticos, artículos de decoración y/o arte que pudieran aparecer en las fotografías. El precio de venta no incluye gastos e impuestos de escrituración, o gastos derivados por algún tipo de credito.'

# Eliminamos la leyenda de la columna 'Título'
df['Título'] = df['Título'].str.replace(leyenda_a_eliminar, '', regex=False)

# Opcional: eliminar espacios extra que puedan quedar al final
df['Título'] = df['Título'].str.strip()

In [29]:
# Extraer "lote en m2"
# El patrón busca: (un grupo de dígitos) seguido de " m² lote"
df['lote_m2'] = df['Características'].str.extract(r'(\d+)\s*m²\s*lote', flags=re.IGNORECASE).astype('Int64')

# Extraer "recamaras"
# El patrón busca: (un grupo de dígitos) seguido de " rec." (el punto es opcional)
df['recamaras'] = df['Características'].str.extract(r'(\d+)\s*rec\.?', flags=re.IGNORECASE).astype('Int64')

# Extraer "baños"
# El patrón busca: (un grupo de dígitos) seguido de " baños"
df['baños'] = df['Características'].str.extract(r'(\d+)\s*baño\s?', flags=re.IGNORECASE).astype('Int64')

# Extraer "estacionamiento"
# El patrón busca: (un grupo de dígitos) seguido de " estac" o " estac."
df['estacionamiento'] = df['Características'].str.extract(r'(\d+)\s*estac\.?', flags=re.IGNORECASE).astype('Int64')


In [30]:
df.estacionamiento.value_counts()

estacionamiento
1     3250
2     3064
3      492
4      200
6       25
5       21
8       13
12       2
22       1
7        1
11       1
13       1
Name: count, dtype: Int64

In [31]:
# por si el estaiconamiento no viene en los detalles básicos pero si en la descripción
# 1. Usar .str.contains() para buscar la palabra "estacionamiento" (ignorando mayúsculas y minúsculas)
# 2. Usar .astype(int) para convertir True/False a 1/0
df['estacionamiento_txt'] = df['Título'].str.contains('estacionamiento', case=False, na=False).astype(int)

In [32]:
# Revisamos el cruce de las columnas de estacionamiento para ver si aporta más información juntarlas y vemos que 
# sí hay reistros donde el estacionamiento estaba vacío e inspeccionando la descripción encontramos más.
pd.crosstab(df['estacionamiento'].fillna(0), df['estacionamiento_txt'])

estacionamiento_txt,0,1
estacionamiento,,
0,1151,1000
1,587,2663
2,469,2595
3,86,406
4,29,171
5,5,16
6,2,23
7,0,1
8,2,11


In [33]:
# Así que complementamos la columna
df['estacionamiento'] = np.where(~df['estacionamiento'].isnull(), df['estacionamiento'], 
                                 np.where((df['estacionamiento'].isnull())&(df['estacionamiento_txt']==1),1,0))

In [34]:
df['estacionamiento'].value_counts()

estacionamiento
1.0     4250
2.0     3064
0.0     1151
3.0      492
4.0      200
6.0       25
5.0       21
8.0       13
12.0       2
22.0       1
7.0        1
11.0       1
13.0       1
Name: count, dtype: int64

In [35]:
# 1. Usar .str.contains() para buscar la palabra "renta" (ignorando mayúsculas y minúsculas)
# 2. Usar .astype(int) para convertir True/False a 1/0
df['en_renta'] = df['Título'].str.contains(' renta ', case=False, na=False).astype(int)

In [36]:
df.en_renta.value_counts()

en_renta
0    4638
1    4584
Name: count, dtype: int64

In [37]:
# 1. Usar .str.contains() para buscar la palabra "venta" (ignorando mayúsculas y minúsculas)
# 2. Usar .astype(int) para convertir True/False a 1/0
df['en_venta'] = df['Título'].str.contains(' venta ', case=False, na=False).astype(int)

In [38]:
df.en_venta.value_counts()

en_venta
0    8925
1     297
Name: count, dtype: int64

In [39]:
pd.crosstab(df.en_renta, df.en_venta)

en_venta,0,1
en_renta,,
0,4520,118
1,4405,179


In [40]:
# para filtrar los departamentos en venta tienen que tener la flag de venta y costo mayor a 1M 
# Y también hay casos en que no ponen un monto de venta pero ponen 1 MXN por poner algo
df = df[(df.en_venta==1)&(df.precio_mxn > 1)&(df.precio_mxn < 1000000)]

In [49]:
df[(df.en_venta==1)&(df.en_renta==0)].sort_values('precio_mxn',ascending=False)

Título  \
2101                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Departamento de Lujo en Venta y Renta, Campos Elíseos, Polanco. Vistas Insuperables al Bosque de Chapultepec. En el corazón de Polanco, sobre la prestigiada avenida Campos Elíseos, se encuentra este exclusivo departamento en el piso 26, un refugio de lujo con espectaculares vistas abiertas hacia el verde infinito del Bosque de Chapultepec. Desde el momento en que cruzas la puerta principal, el espacio transmite una sensación de elegancia atemporal. Los acabados de alta gama, la amplitud de los espacios y la luz natural que inunda cada rincón convierten este departamento en un verdadero oasis en el cielo d

In [42]:
df[(df.en_venta==1)&(df.en_renta==1)]

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn,lote_m2,recamaras,baños,estacionamiento,estacionamiento_txt,en_renta,en_venta
93,Departamento en Renta Amueblado – Ideal para P...,https://www.inmuebles24.com/propiedades/clasif...,"MN 8,000","Betina 25, Lomas Estrella 2da Secc, Iztapalapa...","45 m² lote, 1 rec., 1 baño, 1 estac.",Tláhuac,8000.0,45,1,1,1.0,1,1,1
195,Venta Renta modernos departamentos Residencial...,https://www.inmuebles24.com/propiedades/clasif...,"MN 38,000","Avenida Alfonso Reyes 53\nHipódromo Condesa, C...","2000 m² lote, 2 rec., 2 baños, 2 estac.",Cuauhtémoc,38000.0,2000,2,2,2.0,1,1,1
272,En el corazón de Paseo de la Reforma se alza C...,https://www.inmuebles24.com/propiedades/clasif...,"USD 6,000","Río Ródano 510\nCuauhtémoc, Cuauhtémoc","58 m² lote, 1 rec., 1 baño, 1 estac.",Cuauhtémoc,111960.0,58,1,1,1.0,1,1,1
303,"Departamento exterior en 2o. Nivel, remodelado...",https://www.inmuebles24.com/propiedades/clasif...,"MN 27,600","Calle Bajío\nRoma Sur, Cuauhtémoc","99 m² lote, 2 rec., 2 baños, 2 estac.",Cuauhtémoc,27600.0,99,2,2,2.0,1,1,1
338,Departamento en venta o renta (sin amueblar) u...,https://www.inmuebles24.com/propiedades/clasif...,"MN 38,000","durango 210\nRoma Norte, Cuauhtémoc","133 m² lote, 2 rec., 2 baños, 1 estac.",Cuauhtémoc,38000.0,133,2,2,1.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13061,Este amplio departamento de 360 m² se encuentr...,https://www.inmuebles24.com/propiedades/clasif...,"MN 65,000","loma larga\nLomas de Vista Hermosa, Cuajimalpa...","360 m² lote, 3 rec., 3 baños, 3 estac.",Cuajimalpa de Morelos,65000.0,360,3,3,3.0,1,1,1
13099,¡No dejes pasar esta oportunidad! Característi...,https://www.inmuebles24.com/propiedades/clasif...,"MN 12,400","Monte Carmelo 22, Rosa Torres, Cuajimalpa de M...","64 m² lote, 2 rec., 2 baños, 1 estac.",Cuajimalpa de Morelos,12400.0,64,2,2,1.0,1,1,1
13129,Muy amplio y funcional Penthouse en Campestre ...,https://www.inmuebles24.com/propiedades/clasif...,"MN 68,000","Calle Fresnos No.\nGranjas Palo Alto, Cuajimal...","558 m² lote, 4 rec., 3 baños, 4 estac.",Cuajimalpa de Morelos,68000.0,558,4,3,4.0,0,1,1
13251,Increíble departamento en renta en We Santa Fe...,https://www.inmuebles24.com/propiedades/clasif...,"MN 25,000","Carretera México-Toluca 2971-5392\nLa Rosita, ...","62 m² lote, 1 rec., 1 baño, 1 estac.",Cuajimalpa de Morelos,25000.0,62,1,1,1.0,1,1,1


In [43]:
# 1. Usar .str.contains() para buscar la palabra "amueblado" (ignorando mayúsculas y minúsculas)
# 2. Usar .astype(int) para convertir True/False a 1/0
df['es_amueblado'] = df['Título'].str.contains('amueblado', case=False, na=False).astype(int)

In [44]:
df.es_amueblado.value_counts()

es_amueblado
0    221
1     63
Name: count, dtype: int64

In [45]:
df.isnull().sum()

Título                 0
Enlace                 0
Precio                 0
Dirección              0
Características        0
Municipio de Origen    0
precio_mxn             0
lote_m2                3
recamaras              1
baños                  4
estacionamiento        0
estacionamiento_txt    0
en_renta               0
en_venta               0
es_amueblado           0
dtype: int64

In [46]:
df[df.en_renta == 1].precio_mxn.describe()

count       179.000000
mean      65043.597765
std       46969.912910
min        8000.000000
25%       27300.000000
50%       50000.000000
75%       94150.000000
max      242580.000000
Name: precio_mxn, dtype: float64

In [51]:
pd.set_option('display.max_colwidth', 50)

In [52]:
# Queremos identificar outliers
df.sort_values('precio_mxn', ascending=False)[:10]

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn,lote_m2,recamaras,baños,estacionamiento,estacionamiento_txt,en_renta,en_venta,es_amueblado
635,En venta departamento totalmente amueblado en ...,https://www.inmuebles24.com/propiedades/clasif...,"USD 15,000","Av. Paseo de la Reforma\nCuauhtémoc, Cuauhtémoc","240 m² lote, 2 rec., 2 baños, 2 estac.",Cuauhtémoc,279900.0,240,2,2,2.0,1,0,1,1
2101,"Departamento de Lujo en Venta y Renta, Campos ...",https://www.inmuebles24.com/propiedades/clasif...,"USD 15,000","campos eliseos\nPolanco IV Sección, Polanco","443 m² lote, 2 rec., 2 baños, 5 estac.",Miguel Hidalgo,279900.0,443,2,2,5.0,0,0,1,0
3319,Extraordinario departamento en venta con ampli...,https://www.inmuebles24.com/propiedades/clasif...,"USD 15,000","CAMPOS ELISEOS\nPolanco, Miguel Hidalgo","460 m² lote, 3 rec., 3 baños, 5 estac.",Miguel Hidalgo,279900.0,460,3,3,5.0,0,0,1,0
3757,"Penthouse de Lujo en Venta – Arquímedes, Polan...",https://www.inmuebles24.com/propiedades/clasif...,"USD 13,500","Arquimedes\nPolanco, Miguel Hidalgo","550 m² lote, 4 rec., 4 baños, 3 estac.",Miguel Hidalgo,251910.0,550,4,4,3.0,1,0,1,0
2618,Departamento de Lujo en Renta o Venta Frente a...,https://www.inmuebles24.com/propiedades/clasif...,"USD 13,000","Luis G Urbina\nPolanco, Miguel Hidalgo","325 m² lote, 2 rec., 2 baños, 4 estac.",Miguel Hidalgo,242580.0,325,2,2,4.0,1,1,1,0
2690,Elegante Departamento en Venta cerca del Parqu...,https://www.inmuebles24.com/propiedades/clasif...,"USD 13,000","Campos Elíseos\nPolanco, Miguel Hidalgo","460 m² lote, 3 rec., 3 baños, 2 estac.",Miguel Hidalgo,242580.0,460,3,3,2.0,1,0,1,0
641,"Residencia en Renta – Paseo de la Reforma, con...",https://www.inmuebles24.com/propiedades/clasif...,"USD 13,000","Paseo de la Reforma\nCuauhtémoc, Cuauhtémoc","234 m² lote, 3 rec., 3 baños, 2 estac.",Cuauhtémoc,242580.0,234,3,3,2.0,1,1,1,0
3514,Precioso departamento en renta Y venta en parq...,https://www.inmuebles24.com/propiedades/clasif...,"USD 12,000","LUIS G URBINA\nPolanco, Miguel Hidalgo","325 m² lote, 3 rec., 2 baños, 4 estac.",Miguel Hidalgo,223920.0,325,3,2,4.0,0,1,1,0
3325,Precioso Departamento en venta ó renta en tres...,https://www.inmuebles24.com/propiedades/clasif...,"USD 10,000","tres picos 19\nPolanco V Sección, Polanco","650 m² lote, 3 rec., 3 baños, 4 estac.",Miguel Hidalgo,186600.0,650,3,3,4.0,1,1,1,0
6272,Venta renta departamento bosques de santa fe c...,https://www.inmuebles24.com/propiedades/clasif...,"MN 180,000",Club de Golf Bosques de Santa Fe\nClub de Golf...,"460 m² lote, 3 rec., 3 baños, 4 estac.",Cuajimalpa de Morelos,180000.0,460,3,3,4.0,1,1,1,0


In [53]:
# Dividir la columna 'Dirección' y expandir en nuevas columnas
# Usamos r'[,\n]' como la expresión regular para buscar tanto comas como saltos de línea
df_split = df['Dirección'].str.split(r'[,\n]', expand=True)

In [54]:

# Concatenar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_split], axis=1)

In [57]:
df

,Título,Enlace,Precio,Dirección,Características,Municipio de Origen,precio_mxn,lote_m2,recamaras,baños,...,en_renta,en_venta,es_amueblado,0,1,2,3,4,5,6
93,Departamento en Renta Amueblado – Ideal para P...,https://www.inmuebles24.com/propiedades/clasif...,"MN 8,000","Betina 25, Lomas Estrella 2da Secc, Iztapalapa...","45 m² lote, 1 rec., 1 baño, 1 estac.",Tláhuac,8000.0,45,1,1,...,1,1,1,Betina 25,Lomas Estrella 2da Secc,Iztapalapa,09890 Ciudad de México,Cdmx,Lomas Estrella,Iztapalapa
170,Departamento en Venta – 70 m² – Col. Doctores ...,https://www.inmuebles24.com/propiedades/clasif...,"MN 22,000","Dr. Manuel Martínez\nDoctores, Cuauhtémoc","70 m² lote, 2 rec., 2 baños, 1 estac.",Cuauhtémoc,22000.0,70,2,2,...,0,1,0,Dr. Manuel Martínez,Doctores,Cuauhtémoc,None,None,None,None
195,Venta Renta modernos departamentos Residencial...,https://www.inmuebles24.com/propiedades/clasif...,"MN 38,000","Avenida Alfonso Reyes 53\nHipódromo Condesa, C...","2000 m² lote, 2 rec., 2 baños, 2 estac.",Cuauhtémoc,38000.0,2000,2,2,...,1,1,1,Avenida Alfonso Reyes 53,Hipódromo Condesa,Cuauhtémoc,None,None,None,None
232,"Departamento en venta penthouse, condesa. Pent...",https://www.inmuebles24.com/propiedades/clasif...,"MN 60,000","TAMAULIPAS 261 BIS\nCondesa, Cuauhtémoc","157 m² lote, 2 rec., 2 baños, 2 estac.",Cuauhtémoc,60000.0,157,2,2,...,0,1,0,TAMAULIPAS 261 BIS,Condesa,Cuauhtémoc,None,None,None,None
234,La propiedad en venta se encuentra en la colon...,https://www.inmuebles24.com/propiedades/clasif...,"MN 65,000","San Luis Potosí 44\nRoma Norte, Cuauhtémoc","160 m² lote, 2 rec., 3 baños, 2 estac.",Cuauhtémoc,65000.0,160,2,3,...,0,1,0,San Luis Potosí 44,Roma Norte,Cuauhtémoc,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13116,"En Contadero, este departamento de 100 m² ofre...",https://www.inmuebles24.com/propiedades/clasif...,"MN 27,000","Carretera mexico toluca 5860\nContadero, Cuaji...","100 m² lote, 2 rec., 2 baños, 2 estac.",Cuajimalpa de Morelos,27000.0,100,2,2,...,0,1,0,Carretera mexico toluca 5860,Contadero,Cuajimalpa de Morelos,None,None,None,None
13129,Muy amplio y funcional Penthouse en Campestre ...,https://www.inmuebles24.com/propiedades/clasif...,"MN 68,000","Calle Fresnos No.\nGranjas Palo Alto, Cuajimal...","558 m² lote, 4 rec., 3 baños, 4 estac.",Cuajimalpa de Morelos,68000.0,558,4,3,...,1,1,0,Calle Fresnos No.,Granjas Palo Alto,Cuajimalpa de Morelos,None,None,None,None
13169,"Departamento en Venta – Tres Cumbres, Av. De l...",https://www.inmuebles24.com/propiedades/clasif...,"MN 60,000","Av. De los poetas 89\nSan Mateo Tlaltenango, C...","196 m² lote, 3 rec., 1 baño, 3 estac.",Cuajimalpa de Morelos,60000.0,196,3,1,...,0,1,0,Av. De los poetas 89,San Mateo Tlaltenango,Cuajimalpa de Morelos,None,None,None,None
13251,Increíble departamento en renta en We Santa Fe...,https://www.inmuebles24.com/propiedades/clasif...,"MN 25,000","Carretera México-Toluca 2971-5392\nLa Rosita, ...","62 m² lote, 1 rec., 1 baño, 1 estac.",Cuajimalpa de Morelos,25000.0,62,1,1,...,1,1,0,Carretera México-Toluca 2971-5392,La Rosita,Cuajimalpa de Morelos,None,None,None,None
